In [1]:
import numpy as np
import pandas as pd
import skimage.filters as filters
from functools import partial, lru_cache
from typing import Union
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [2]:
import os, re, math
import h5py

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
#sample_data = pd.read_pickle("./data/sample_sc_data.pkl")
sample_data = pd.read_hdf("/content/drive/MyDrive/immunolinguistics/frames/ZZZU.h5", start=499608-1000, stop=499608)
sample_data.head(n=10)

,CD123,CD14,CD11c,MHCII
498608,-0.819428,0.599702,0.076553,1.103783
498609,0.718618,0.115962,1.027351,0.263494
498610,-0.157862,-0.576694,-0.376348,-0.093487
498611,0.285110,-1.159225,-0.029846,-0.125228
498612,-0.157862,-0.831927,0.014004,-0.173780
498613,-0.465809,-0.675357,0.621934,0.000000
498614,-7.088847,-5.474100,-0.237629,-2.562109
498615,0.813498,-0.046108,0.923564,0.092588
498616,0.092118,-0.095462,-1.048623,-0.326603
498617,-5.074555,-1.392902,-0.536434,-2.055866


# Binary Encodoing
This strategy uses Otsu's thresholding to define + and - for each cell based on the maximum in-class homogeneity. The input is a matrix of $num\_obs \times num\_features$

In [14]:
def threshold_val(value, threshold):
    return 0 if value <= threshold else 1    
    
def remove_outliers(x, percentiles=[5, 95]):
    a = np.array(x)
    upper_percentile = np.percentile(a, percentiles[1])
    lower_percentile = np.percentile(a, percentiles[0])
    mask = np.logical_and(a < upper_percentile, a > lower_percentile)
    return a[mask]

    
    
def vocab_generator(df: pd.DataFrame, style='discrete'):
    """
    For every gene, generate a 'word' that is the ambiguous gene (e.g. "CD45"), 
    the positive, and the negative genes (e.g. "CD45+" and "CD45-"). 
    
    The "style" parameter is meant to let this be expandable further on.
    
    :param df: The data frame containing the `obs x genes` matrix
    :param style: the style of vocab to generate. Options:
    :yield: the ambiguous, negative, and positive versions of that gene.  
    """
    
    
    if style.upper() == 'DISCRETE':
        for val in iter(df.columns):
            val = val.strip().lower()
            yield val, val + "-", val + "+"
                    
def generate_vocabulary(df, style='discrete'):
    generator = vocab_generator(df=df, style=style)
    vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
    if style.upper() == 'DISCRETE':
        for tup in generator:
            [vocab.append(v) for v in tup]
    return vocab
    
def binary_encode(df, method='otsu', histogram_bins=1000, clip_data=False):
    """
    :param df: Data frame with one row per cell and one column per gene.
    :returns: Encoded table-- a copy of df with each numeric entry replaced with 1 for + and 0 for -
    :returns: Vocabulary-- a comprehensive vocabulary of each gene in the table
    :returns: Gene thresholds-- a dictionary relating {Gene: threshold}
    """
    gene_thresh_dict = {}
    encoded_table = pd.DataFrame(0, index=df.index, columns=df.columns)
    
    # Go through each columns and 
    # TODO: subsample to avoid loading 1 billion cells at once
    for (col_name, col_data) in df.iteritems():
        
        # Remove outliers from data if parameter is passed (makes more of a difference in Otsu's method)
        if clip_data:
            data_values = remove_outliers(col_data.values)
        else:
            data_values = col_data.values
        
        # Determine threshold based on method passed.
        if method.upper() == "OTSU":
            thresh = filters.threshold_otsu(data_values, nbins=histogram_bins)
        elif method.upper() == "MEDIAN":
            thresh = np.median(data_values)
        else:
            raise ValueError("Unexpected value for parameter `method`: %s" % method)
            
        # Set value in threhsold disctionary, encode table.
        gene_thresh_dict[col_name] = thresh
        threshold_partial = partial(threshold_val, threshold=thresh)
        encoded_table.loc[:, col_name] = df.loc[:, col_name].apply(threshold_partial)
    
    vocab = generate_vocabulary(df, style='discrete')
    
    return encoded_table, gene_thresh_dict, vocab
        
        

In [15]:
encoded_table_otsu, thresh_dict_otsu, vocabulary_otsu = binary_encode(sample_data, method='otsu')
encoded_table_median, thresh_dict_median, vocabulary_median = binary_encode(sample_data, method='median')

print()

In [16]:
def write_vocabulary_file(fp, vocab_list, sort=False):
    if sort:
        vocab_list = sorted(vocab_list)
        
    with open(fp, 'w') as output:
        for word in vocab_list:
            output.write(word + "\n")

In [17]:
#!pwd
write_vocabulary_file("vocab.txt", vocabulary_otsu)
! head vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
cd123
cd123-
cd123+
cd14
cd14-


In [18]:
@lru_cache(maxsize=1024)
def binary_to_text(value, protein_name):
    protein_name = protein_name.lower()
    return protein_name + "+" if value == 1 else protein_name + "-"
    
def binary_table_to_text(df):
    text_table = pd.DataFrame(0, index=df.index, columns=df.columns)
    for col in df.columns:
        bin2text_partial = partial(binary_to_text, protein_name=col)
        text_table.loc[:, col] = df.loc[:, col].apply(bin2text_partial)
    return text_table
    

In [19]:
encoded_table_otsu

,CD123,CD14,CD11c,MHCII
498608,1,1,1,1
498609,1,1,1,1
498610,1,1,1,1
498611,1,1,1,1
498612,1,1,1,1
...,...,...,...,...
499603,1,1,1,1
499604,1,1,1,1
499605,1,1,1,1
499606,1,1,1,1


In [20]:
binary_table_to_text(encoded_table_otsu)

,CD123,CD14,CD11c,MHCII
498608,cd123+,cd14+,cd11c+,mhcii+
498609,cd123+,cd14+,cd11c+,mhcii+
498610,cd123+,cd14+,cd11c+,mhcii+
498611,cd123+,cd14+,cd11c+,mhcii+
498612,cd123+,cd14+,cd11c+,mhcii+
...,...,...,...,...
499603,cd123+,cd14+,cd11c+,mhcii+
499604,cd123+,cd14+,cd11c+,mhcii+
499605,cd123+,cd14+,cd11c+,mhcii+
499606,cd123+,cd14+,cd11c+,mhcii+


In [21]:
def create_names(name, n_grades):
    """
    Given a protein number and the number of grades, return a list of [protein::grade]
    Example:
    >>> create_names("CD45", 4)
    ["CD45::0", "CD45::1", "CD45::2", "CD45::3"]
    
    """
    
    grades = [name + "::" + str(grade) for grade in range(n_grades)]
    return grades
        

def graded_encode(df, n_comps=3, n_samples=-1, method='gmm'):
    """
    Gaussian Mixture Model
    Kmeans
    Quantiles = [25, 50, 75]
    """
    
    gene_thresh_dict = {}
    encoded_table = pd.DataFrame(0, index=df.index, columns=df.columns)

    
    for (col_name, col_data) in df.iteritems():
        reshaped_data = col_data.values.reshape(-1, 1)
        gmm = GaussianMixture(n_components=n_comps, covariance_type='full', verbose=0)
        preds = gmm.fit_predict(reshaped_data)
        grades = create_names(col_name, n_comps)
        
        # Find the indices for sorting the models in increasing order (min expr to max expr)
        original_idxs = np.arange(n_comps)
        sorted_idxs = np.argsort(gmm.means_.ravel())
        
      
        # This is a swap dictionary relating the mean to the sorted position (i.e. grade)
        swap_dict = {original: new for original, new in zip(original_idxs.ravel(), sorted_idxs.ravel())}
        
        graded_genes = []
        for i in preds:
            graded_genes.append(swap_dict[i])
            
        #graded_genes = np.vectorize(swap_dict.get)(preds)
        #print(graded_genes)
        encoded_table.loc[:, col_name] = np.asarray(graded_genes)
        
        
    return encoded_table
    

In [22]:
graded_table = graded_encode(sample_data)

In [23]:
@lru_cache(maxsize=1024)
def graded_to_text(value, protein_name):
    return protein_name + "::" + str(value)
    
def graded_table_to_text(df):
    text_table = pd.DataFrame(0, index=df.index, columns=df.columns, dtype=str)
    for col in df.columns:
        graded2text_partial = partial(graded_to_text, protein_name=col)
        text_table.loc[:, col] = df.loc[:, col].apply(graded2text_partial)
    return text_table
    
    

In [24]:
sample_data.head()

,CD123,CD14,CD11c,MHCII
498608,-0.819428,0.599702,0.076553,1.103783
498609,0.718618,0.115962,1.027351,0.263494
498610,-0.157862,-0.576694,-0.376348,-0.093487
498611,0.285110,-1.159225,-0.029846,-0.125228
498612,-0.157862,-0.831927,0.014004,-0.173780


In [25]:
graded_table.head()

,CD123,CD14,CD11c,MHCII
498608,1,1,2,2
498609,1,1,0,1
498610,1,1,2,1
498611,1,0,2,1
498612,1,0,2,1


In [26]:
graded_table_to_text(graded_table.head())

,CD123,CD14,CD11c,MHCII
498608,CD123::1,CD14::1,CD11c::2,MHCII::2
498609,CD123::1,CD14::1,CD11c::0,MHCII::1
498610,CD123::1,CD14::1,CD11c::2,MHCII::1
498611,CD123::1,CD14::0,CD11c::2,MHCII::1
498612,CD123::1,CD14::0,CD11c::2,MHCII::1


In [31]:
output_folder = '/content/drive/MyDrive/immunolinguistics/frames_converted/'
#!mkdir /content/drive/MyDrive/immunolinguistics/frames_converted/
raw_data_folder = '/content/drive/MyDrive/immunolinguistics/frames/'
raw_data_folder = '/content/drive/MyDrive/immunolinguistics/split_frames/smaller/'
files = os.listdir(raw_data_folder)
files

['Z238_0.pkl',
 'Z238_2.pkl',
 'Z238_1.pkl',
 'Z238_3.pkl',
 'Z238_4.pkl',
 'Z238_6.pkl',
 'Z238_5.pkl',
 'Z238_7.pkl',
 'Z238_8.pkl',
 'Z238_9.pkl',
 'Z238_10.pkl',
 'Z238_11.pkl',
 'Z238_12.pkl']

In [50]:
def converPKL2text(file, data_folder, output_prefix, output_folder, encode_method = "otsu", max_lines = int(1e6)):
    if(data_folder == output_folder):
        print("Error: input file folder should not be same as output folder!\n")
        return(None)
    path_to_infile = data_folder + "/" + file
    print("Processing ", path_to_infile, " with method", encode_method, "...", end = "")
    if not os.path.isfile(path_to_infile):
        return 1

    if bool(re.search(".pkl$", file)):
        data = pd.read_pickle(path_to_infile)
        print("Data have ", data.shape[0], " lines")
        na_cnt = data.isnull().sum(axis = 0)
        if na_cnt.max() > 0:
            print(path_to_infile, " has some NAs. Skip...")
            print(na_cnt);
            return 1;
        
        if data.shape[0] <= max_lines:
            path_to_output_file = output_folder + "/" + output_prefix + ".txt.gz"
            path_to_output_voc_file = output_folder + "/" + output_prefix + ".voc.txt"
            if os.path.exists(path_to_output_file):
               print("File ", path_to_output_file, "exists, skip...")
                #return 1
            if encode_method == "gmm":
                graded_table = graded_encode(data)
                text_table = graded_table_to_text(graded_table)
                #vocabulary = generate_vocabulary(text_table) ###
            else:
                encoded_table, thresh_dict, vocabulary = binary_encode(data, method= encode_method)
                text_table = binary_table_to_text(encoded_table)
            #pd.to_pickle(text_table, path_to_output_file)
            write_vocabulary_file(path_to_output_voc_file, vocabulary)  
            text_table.to_csv(path_to_output_file, sep=' ', index=True, header=False, compression = "gzip") ###
        else:
            print("encoding ", max_lines, "lines per time", end = "")
            for k in range(0, math.ceil(data.shape[0]/max_lines)):
                print(k, "...", end = "");
                path_to_output_file = output_folder + "/" + output_prefix + "." + str(k) + ".txt.gz"
                path_to_output_voc_file = output_folder + "/" + output_prefix + "." + str(k) + ".voc.txt"
                #if os.path.exists(path_to_output_file):
                #  print("File ", path_to_output_file, "exists, skip...")
                #  continue
                print("\n output", path_to_output_file, end = ' ')
                start = k * max_lines;
                end = (k+1) * max_lines;
                if(end > data.shape[0] or data.shape[0] - end < max_lines/2):
                    end = data.shape[0]
                print(" start:", start, "end:", end, end = "\n")
                sub_data = data.iloc[start:end,]
                if encode_method == "gmm":
                   graded_table = graded_encode(sub_data)
                   text_table = graded_table_to_text(graded_table)
                else:
                   encoded_table, thresh_dict, vocabulary = binary_encode(sub_data, method= encode_method)
                   text_table = binary_table_to_text(encoded_table)
                #pd.to_pickle(text_table, path_to_output_file) 
                write_vocabulary_file(path_to_output_voc_file, vocabulary)
                text_table.to_csv(path_to_output_file, sep=' ', index=True, header=False, compression = "gzip") ###    
                if end >= data.shape[0]:
                    return 1
    elif bool(re.search(".hd5$", file)):
        print("skip hd5 files now...")
        return 1
        k = 0
        while 1:
            data = pd.read_hdf(path_to_infile, start = k*max_lines, stop = (k+1)*max_lines)
            k+=1
    else:
        print("Unknown file format, skip...\n")
        return 1
    print(" ... finished!\n")
    return 1

In [ ]:
output_folder = '/content/drive/MyDrive/immunolinguistics/frames_converted/'
#!mkdir /content/drive/MyDrive/immunolinguistics/frames_converted/
raw_data_folder = '/content/drive/MyDrive/immunolinguistics/frames/'
#raw_data_folder = '/content/drive/MyDrive/immunolinguistics/split_frames'
files = os.listdir(raw_data_folder)
files

#for file in files:
#files = ['Z2ZJ.pkl']
for i in files:
  if i == "Z238.pkl": continue # file size too large, skip this one. Procee its splitted data.
  for m in ['otsu', 'median']:
    file_prefix = i.split(".")[0]
    file_prefix = file_prefix + '-' + m
    converPKL2text(i, raw_data_folder, file_prefix, output_folder, encode_method= m)

Processing  /content/drive/MyDrive/immunolinguistics/frames//Z2L2.pkl  with method otsu ...Data have  8817845  lines
encoding  1000000 lines per time0 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.0.txt.gz  start: 0 end: 1000000
1 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.1.txt.gz  start: 1000000 end: 2000000
2 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.2.txt.gz  start: 2000000 end: 3000000
3 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.3.txt.gz  start: 3000000 end: 4000000
4 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.4.txt.gz  start: 4000000 end: 5000000
5 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.5.txt.gz  start: 5000000 end: 6000000
6 ...
 output /content/drive/MyDrive/immunolinguistics/frames_converted//Z2L2-otsu.6.txt.gz  start: 6000000 end: 7000000
7 ...
 out

In [ ]:
output_folder = '/content/drive/MyDrive/immunolinguistics/frames_converted/'
#!mkdir /content/drive/MyDrive/immunolinguistics/frames_converted/
#raw_data_folder = '/content/drive/MyDrive/immunolinguistics/frames/'
raw_data_folder = '/content/drive/MyDrive/immunolinguistics/split_frames'
files = os.listdir(raw_data_folder)
files

#for file in files:
#files = ['Z2ZJ.pkl']
for i in files:
  #if(i == "Z238.pkl"): continue
  if i == "ZZBG.pkl": continue
  for m in ['otsu', 'median']:    
    file_prefix = i.split(".")[0]
    file_prefix = file_prefix + '-' + m
    converPKL2text(i, raw_data_folder, file_prefix, output_folder, encode_method= m)

In [44]:
!gzip -dc /content/drive/MyDrive/immunolinguistics/frames_converted/Z2L2-otsu.0.txt.gz |head

0 cd45+ hla-dr+ cd27+ cd235+ cd19- cd5- cd38+ cd4+ cd8- cd20- cd16+ cd123- cd56+ cd14+ cd11c+ cd45ra+ cd3- cd66+ cd61+
1 cd45- hla-dr+ cd27- cd235+ cd19+ cd5- cd38- cd4- cd8- cd20- cd16- cd123- cd56- cd14- cd11c- cd45ra+ cd3- cd66- cd61-
2 cd45+ hla-dr+ cd27- cd235+ cd19+ cd5- cd38- cd4- cd8- cd20- cd16- cd123- cd56- cd14- cd11c- cd45ra+ cd3- cd66+ cd61-
3 cd45+ hla-dr+ cd27- cd235- cd19+ cd5- cd38+ cd4- cd8- cd20+ cd16- cd123+ cd56+ cd14- cd11c- cd45ra+ cd3- cd66- cd61-
4 cd45+ hla-dr+ cd27+ cd235- cd19- cd5+ cd38+ cd4- cd8- cd20- cd16- cd123- cd56+ cd14- cd11c- cd45ra+ cd3- cd66+ cd61-
5 cd45+ hla-dr- cd27+ cd235+ cd19- cd5+ cd38+ cd4- cd8+ cd20- cd16+ cd123- cd56+ cd14- cd11c+ cd45ra+ cd3+ cd66+ cd61+
6 cd45+ hla-dr+ cd27+ cd235- cd19+ cd5+ cd38- cd4- cd8- cd20- cd16- cd123- cd56- cd14- cd11c- cd45ra- cd3- cd66+ cd61-
7 cd45+ hla-dr- cd27- cd235+ cd19- cd5- cd38+ cd4- cd8- cd20- cd16+ cd123- cd56+ cd14- cd11c+ cd45ra+ cd3- cd66- cd61-
8 cd45+ hla-dr+ cd27+ cd235+ cd19+ cd5+ cd38- cd

In [45]:
x = pd.read_pickle("/content/drive/MyDrive/immunolinguistics/frames/ZZBG.pkl")

In [48]:
x.isnull().sum(axis = 0).max() > 0

True

In [50]:
x.shape

(4212900, 7)

In [12]:
'CD45+'.lower()

'cd45+'

In [49]:
x.isnull().sum(axis = 0)

CD95            0
CD27      1049609
CD3       3163291
CD4             0
CD45RA          0
CD25            0
CD8             0
dtype: int64